In [ ]:
%matplotlib inline

# Climatology

As a first step in coordinate development, we'll work on sections of climatology from [WOA13](https://www.nodc.noaa.gov/OC5/woa13/). Because temperature is given in-situ, we first have to convert to potential temperature. Similar to [convert_WOA13](https://github.com/adcroft/convert_WOA13), we use the Python `gsw` package, which implements TEOS-10.

In [ ]:
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
from remapping import mom_remapping
import gsw

As an initial set of input files, we load up THREDDS URLs for monthly averaged climatologies from 2005-2012.

In [ ]:
url_format = 'https://data.nodc.noaa.gov/thredds/dodsC/woa/WOA13/DATAv2/{}/netcdf/A5B2/1.00/woa13_A5B2_{}{:02}_01v2.nc'

In [ ]:
temp_urls = [url_format.format('temperature', 't', i+1) for i in range(12)]
salt_urls = [url_format.format('salinity', 's', i+1) for i in range(12)]

Now we define a function that gives us a potential density section from a given month (default January), and range of latitudes and longitudes.

In [ ]:
def sect(lat, lon, month=0):
    salt_data = Dataset(salt_urls[month])
    temp_data = Dataset(temp_urls[month])
    
    depth = salt_data.variables['depth'][:]
    # calculate indices for lat/lon
    lat_d = salt_data.variables['lat'][:]
    lon_d = salt_data.variables['lon'][:]
    lat_i = (lat.min() <= lat_d) & (lat_d <= lat.max())
    lon_i = (lon.min() <= lon_d) & (lon_d <= lon.max())
    
    # compute absolute salinity from practical salinity
    sp = salt_data.variables['s_an'][0,:,lat_i,lon_i]
    print(sp.shape)
    sa = gsw.SA_from_SP(sp, depth, lon, lat)
    print(sa.shape, sa)
    
    # compute conservative temperature from in-situ temperature and absolute salinity
    ct = gsw.CT_from_t(sa, temp_data.variables['t_an'][0,:,lat_i,lon_i], depth)
    
    salt_data.close()
    temp_data.close()
    
    # compute potential density wrt surface
    return gsw.rho(sa, ct, 0)

# Sections

Although our choice of coordinate should apply globally, we're particularly interested in a few troublesome spots, where there tend to always be problems, such as the Denmark Strait and the Sulu Sea. We may also care about dense overflows off Antarctica.

## Denmark Strait
Because it's pretty easy to find, we'll look at the Denmark strait from 22.5 to 39.5 degrees West, at 63.5 degrees North.

In [ ]:
temp = Dataset('data/woa13_A5B2_t01_01v2.nc', 'r')
salt = Dataset('data/woa13_A5B2_s01_01v2.nc', 'r')

In [ ]:
lat = temp.variables['lat'][:]
lon = temp.variables['lon'][:]
dep = temp.variables['depth'][:]

In [ ]:
lon_i = (-39.5 <= lon) & (lon <= -22.5)

t_sect = temp.variables['t_an'][0,:,lat==63.5,lon_i].squeeze()
s_sect = salt.variables['s_an'][0,:,lat==63.5,lon_i].squeeze()
lon_sect = lon[lon_i]

Using TEOS-10, we can convert from practical salinity to absolute salinity, and from in-situ temperature to conservative temperature. From here, we can compute the locally-referenced density, and the potential density referenced to 2000m.

In [ ]:
sa_sect = np.empty_like(s_sect)
ct_sect = np.empty_like(t_sect)
rho_sect = np.empty_like(s_sect)
rhop_sect = np.empty_like(rho_sect)

for i in range(s_sect.shape[1]):
    sa_sect[:,i] = gsw.SA_from_SP(s_sect[:,i], dep, -39.5 + i, 63.5)
    ct_sect[:,i] = gsw.CT_from_t(sa_sect[:,i], t_sect[:,i], dep)
    rho_sect[:,i] = gsw.rho(sa_sect[:,i], ct_sect[:,i], dep)
    rhop_sect[:,i] = gsw.rho(sa_sect[:,i], ct_sect[:,i], 2000)

Let's take a look at the two density sections. The locally-refenced density shows very linear stratification (this is probably expected?), whereas the potential density shows a much clearer mixed layer (also as expected).

In [ ]:
ax = plt.subplot(121)
plt.pcolormesh(lon_sect, dep, rho_sect)
ax.invert_yaxis()
#plt.colorbar()

ax = plt.subplot(122)
plt.pcolormesh(lon_sect, dep, rhop_sect)
ax.invert_yaxis()
plt.colorbar()

We're looking at January, so this should be a Winter mixed layer. It's around 100m deep in the shallower regions, but nearly up to 400m deep in the deeper parts of the middle of the strait.

In [ ]:
for i in range(rhop_sect.shape[1]):
    plt.plot(rhop_sect[:,i], dep, '*')
plt.gca().invert_yaxis()
plt.grid(axis='y')

In [ ]:
# mixed layer depth for each column
mld = np.array([50, 50, 100, 200, 200, 200, 200, 200, 200, 200, 300, 300, 350, 350, 300, 100, 100, 100])

To give some information between water columns, we want to compute neutral density differences. We can use the thermal expansion and haline contraction coefficients from the mean salinity, temperature and pressure of the water parcels, as well as the local density from the mean.

In [ ]:
rhop_sect[30,5:7]

In [ ]:
sa_c = (sa_sect[30,5] + sa_sect[30,6]) / 2
ct_c = (ct_sect[30,5] + ct_sect[30,6]) / 2

In [ ]:
rho_c, alpha_c, beta_c = gsw.rho_alpha_beta(sa_c, ct_c, dep[30])

In [ ]:
nd_diff = rho_c * (beta_c * (sa_sect[30,5] - sa_sect[30,6]) - alpha_c * (ct_sect[30,5] - ct_sect[30,6]))
print(nd_diff)

Now we want to compute a distance (in physical space) by which to shift the parcel in order to flatten out the difference.

In [ ]:
g = 9.7963 # value of gravity used in gsw

In [ ]:
n2 = gsw.Nsquared(sa_sect[29:31,5],
                  ct_sect[29:31,5],
                  dep[29:31])
dz = (g**2 * nd_diff) / (1e4 * n2[0])
print(dz)

# Grid Generation

We can take our initial grid (which is set by the depths of observations, some of which are masked) and convert that to a representative model grid with millimetre-thick layers instead of completely missing layers.

In [ ]:
# depths of values with local mask
gr_dep = np.ma.array(np.tile(dep.reshape(-1, 1), (1, t_sect.shape[1])), mask=t_sect.mask)

# deepest data within each column
topo = gr_dep.max(axis=0).compressed().reshape(1,-1)

# interface positions between valid data
gr_int = (gr_dep[1:] + gr_dep[:-1]) / 2
m = gr_int.mask
gr_int = gr_int.filled()

# fill masks within each column with topo
# add a millimetre for each successive masked value
# so we don't have anything with exactly zero thickness
gr_int[m] = (np.tile(topo, (gr_int.shape[0], 1)) + 1e-3 * np.cumsum(m, axis=0))[m]

# prepend 0 to each column, append topo
gr_int = np.concatenate((np.zeros((1, gr_int.shape[1])), gr_int, topo), axis=0)
gr_lay = (gr_int[1:,:] + gr_int[:-1,:]) / 2

# calculate thickness
gr_h   = np.diff(gr_int, axis=0)

We can go about actually generating the grid from the data in a couple of different ways:

The first way is to use the Burchard and Beckers (2004) vertical grid diffusion with a coefficient representing the mixed layer region, and another representing the adjustment required to minimise local neutral density curvature.

The second way is more along the lines of Hofmeister et al. (2010). This still involves a vertical grid diffusion, but we're not using any of the terms in this equation, so perhaps we should ignore it. Then we're left with the isopycnal (or in our case, neutral density difference) tendency term. Because this may lead to invalid layers, we follow up with a corrective step that's weighted in order to preserve resolution in the mixed layer.

## Neutral Density Difference Tendency

Using ghost cells, we can calculate the neutral density difference for every cell. Not sure what to do about vanished cells, but for the moment we'll just give them the same value as the first non-vanished cell above them in the same column.

In [ ]:
m = sa_sect.mask
bot_i = np.cumsum(~m, axis=0).max(axis=0) - 1

# absolute salinity for remapping
# get bottom salinity for each column
sa_bot = sa_sect[bot_i,np.arange(sa_sect.shape[1])].compressed().reshape(1,-1)
sa_map = sa_sect.filled() # un-mask array
sa_map[m] = np.tile(sa_bot, (sa_sect.shape[0], 1))[m]

sa_int = (sa_map[1:,:] + sa_map[:-1,:]) / 2
# add ghost columns
sa_gst = np.concatenate([sa_int[:,1,np.newaxis], sa_int, sa_int[:,-2,np.newaxis]], axis=1)

# conservative temperature
ct_bot = ct_sect[bot_i,np.arange(ct_sect.shape[1])].compressed().reshape(1,-1)
ct_map = ct_sect.filled()
ct_map[m] = np.tile(ct_bot, (ct_sect.shape[0], 1))[m]

ct_int = (ct_map[1:,:] + ct_map[:-1,:]) / 2
# add ghost columns
ct_gst = np.concatenate([ct_int[:,1,np.newaxis], ct_int, ct_int[:,-2,np.newaxis]], axis=1)

In [ ]:
# internal interface positions with ghosts
int_gst = np.concatenate([gr_int[1:-1,1,np.newaxis], gr_int[1:-1,:], gr_int[1:-1,-2,np.newaxis]], axis=1)
# neutral density curvature *on interfaces*
ndd = np.empty_like(gr_int[1:-1,:])

for i in range(sa_map.shape[1]):
    # calculate difference to column at left
    sa_c = sa_gst[:,[i,i+1]].mean(axis=1)
    ct_c = ct_gst[:,[i,i+1]].mean(axis=1)
    z_c  = int_gst[:,[i,i+1]].mean(axis=1)
    
    r, a, b = gsw.rho_alpha_beta(sa_c, ct_c, z_c)
    ndd_l = r * (b * (sa_gst[:,i+1] - sa_gst[:,i]) - a * (ct_gst[:,i+1] - ct_gst[:,i]))
    
    # calculate difference to column at right
    sa_c = sa_gst[:,[i+2,i+1]].mean(axis=1)
    ct_c = ct_gst[:,[i+2,i+1]].mean(axis=1)
    z_c  = lay_gst[:,[i+2,i+1]].mean(axis=1)
    
    r, a, b = gsw.rho_alpha_beta(sa_c, ct_c, z_c)
    ndd_r = r * (b * (sa_gst[:,i+2] - sa_gst[:,i+1]) - a * (ct_gst[:,i+2] - ct_gst[:,i+1]))
    
    ndd[:,i] = ndd_r - ndd_l

In [ ]:
ax = plt.subplot(211)
plt.pcolor(lon_sect, gr_int[1:-1], ndd)
ax.invert_yaxis()
plt.colorbar()

ax = plt.subplot(212)
plt.pcolor(lon_sect, gr_int[1:-1], np.sign(ndd))
ax.invert_yaxis()
plt.colorbar()

Now, if the local neutral density curvature `ndd` is positive, we are less dense than we should be, so we need to go deeper (i.e. positive `dz`), and vice versa for a negative curvature. We can get a measure of local $\partial_z\rho_\text{local}$ (for your favourite choice of local density measurement) from $N^2$, which we can calculate with `gsw.Nsquared`. If an 

In [ ]:
n2 = gsw.Nsquared(sa_map, ct_map, gr_lay)[0] # ignore mid-pressure return value
n2[n2 <= 0] = 1e-10 # buoyancy frequency in unstratified/unstable
dz = (g**2 * ndd) / (1e4 * n2)

In [ ]:
ax = plt.subplot(211)
plt.pcolor(lon_sect, int_gst[:,1:-1], np.log10(n2))
ax.invert_yaxis()
plt.colorbar()

ax = plt.subplot(212)
plt.pcolor(lon_sect, gr_lay, np.log(np.abs(dz)))
ax.invert_yaxis()
plt.colorbar()

To make sure we don't get any weird oscillations of the coordinate on the (horizontal) grid scale, we limit the change in interface position to half the distance to the closest adjacent interface.